### Обучение пайплайна

1. Загрузим данные https://www.kaggle.com/shivamb/real-or-fake-fake-jobposting-prediction
2. Соберем пайплайн с простейшим препроцессингом (tfidf) на текстовых данных
3. Обучим логистическую регрессию и сохраним на диск предобученный пайплайн

In [2]:
# for installing DILL
#!pip install dill

In [8]:
!pip install scipy

In [4]:
!pip install eli5

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-project 0.10.2 requires ruamel-yaml, which is not installed.
jupyter-server 1.13.5 requires pywinpty<2; os_name == "nt", but you have pywinpty 2.0.2 which is incompatible.


  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107748 sha256=9c55a8be3be371d90ccac67c8a8055b10355c0cfa1fbc56327050745ce1f118e
  Stored in directory: c:\users\alex\appdata\local\pip\cache\wheels\7b\26\a5\8460416695a992a2966b41caa5338e5e7fcea98c9d032d055c
Successfully built eli5
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3


In [9]:
import pandas as pd
import dill
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import roc_auc_score,roc_curve,scorer
from sklearn.metrics import f1_score

#working with text
from sklearn.feature_extraction.text import TfidfVectorizer

#normalizing data
from sklearn.preprocessing import StandardScaler

#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import precision_score,recall_score

#imputer
from sklearn.impute import SimpleImputer

import sklearn.datasets

ImportError: cannot import name 'scorer' from 'sklearn.metrics' (C:\Users\Alex\DevProgs\Anaconda\lib\site-packages\sklearn\metrics\__init__.py)

Загрузим данные

In [ ]:
df = pd.read_csv("./fake_job_postings.csv")
df.head(3)

Разделим данные на train/test и сохраним тестовую выборку на диск (здесь мы ее касаться уже не будем)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(df, 
                                                    df['fraudulent'], test_size=0.33, random_state=42)
#save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)
#save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

In [4]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class TextImputer(BaseEstimator, TransformerMixin):
    def __init__(self, key, value):
        self.key = key
        self.value = value
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X[self.key] = X[self.key].fillna(self.value)
        return X

In [5]:
features = ['description', 'company_profile', 'benefits']
target = 'fraudulent'

Соберем кусок, ответственный за feature engineering

In [10]:
#combine
description = Pipeline([
                ('imputer', TextImputer('description', '')),
                ('selector', ColumnSelector(key='description')),
                ('tfidf', TfidfVectorizer(max_df=0.9, min_df=10))
            ])
company_profile = Pipeline([
                ('imputer', TextImputer('company_profile', '')),
                ('selector', ColumnSelector(key='company_profile')),
                ('tfidf', TfidfVectorizer(max_df=0.9, min_df=10))
            ])
benefits = Pipeline([
                ('imputer', TextImputer('benefits', '')),
                ('selector', ColumnSelector(key='benefits')),
                ('tfidf', TfidfVectorizer(max_df=0.9, min_df=10))
            ])

feats = FeatureUnion([('description', description),
                      ('company_profile', company_profile),
                      ('benefits', benefits)])

Добавим простейший классификатор

In [11]:
%%time

pipeline = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression()),
])

pipeline.fit(X_train, y_train)

KeyError: 'description'

Посмотрим, как выглядит наш pipeline

In [16]:
pipeline.steps

[('features', FeatureUnion(transformer_list=[('description',
                                  Pipeline(steps=[('imputer',
                                                   TextImputer(key='description',
                                                               value='')),
                                                  ('selector',
                                                   ColumnSelector(key='description')),
                                                  ('tfidf',
                                                   TfidfVectorizer(max_df=0.9,
                                                                   min_df=10))])),
                                 ('company_profile',
                                  Pipeline(steps=[('imputer',
                                                   TextImputer(key='company_profile',
                                                               value='')),
                                                  ('selector',
         

Сохраним модель (пайплайн)

In [20]:
with open("logreg_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)